# Data Science Challenge

In [2]:
import pandas as pd
import numpy as np

# Exercise 1

In [3]:
!ls /home/dsc/Data/challenge/

bookings.csv.bz2  searches.csv.bz2


In [4]:
!bzcat /home/dsc/Data/challenge/bookings.csv.bz2 | head -10000 > bookings_sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/bookings.csv.bz2, output file = (stdout)


In [5]:
!bzip2 -f bookings_sample.csv

In [6]:
!ls -l

total 536
-rw-rw-r-- 1 dsc dsc 535893 nov 20 18:29 bookings_sample.csv.bz2
-rw-rw-r-- 1 dsc dsc   8260 nov 20 18:29 DS_challenge.ipynb


In [7]:
!bzcat bookings_sample.csv.bz2 | wc -l

10000


In [8]:
import bz2
with bz2.BZ2File('/home/dsc/DS/DS_challenge/bookings_sample.csv.bz2') as file_bz2:
    count = 0
    for line in file_bz2:
        count += 1
print(count)

10000


# Exercise 2

In [9]:
bookings = pd.read_csv('/home/dsc/DS/DS_challenge/bookings_sample.csv.bz2', compression='bz2', sep='^')

In [10]:
bookings.shape

(9999, 38)

In [11]:
bookings.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [12]:
bookings.groupby('arr_port')['pax'].sum().sort_values(ascending=False).head(10)

arr_port
HKG         112
LGA          95
ORD          94
JFK          92
SFO          91
LAX          91
MCO          90
DCA          82
DEN          79
LHR          76
Name: pax, dtype: int64

In [14]:
bookings.groupby('arr_port')['pax', 'arr_city'].agg({'pax': 'sum'}).head(10)

,pax
,pax
arr_port,
AAL,0
ABQ,12
ABV,2
ABZ,-6
ACC,4
ADD,7
ADZ,2
AEP,7


**Removing white spaces from the columns**

In [ ]:
bookings.columns = bookings.columns.str.strip()

In [18]:
bookings.columns

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

**Unique values for "rloc" column"**

In [21]:
len(bookings['rloc'].unique())

3638

# Exercise 3